In [3]:
import geopandas
import json
import numpy as np
import pandas as pd
import math

In [16]:
mk_municipalities = geopandas.read_file("https://raw.githubusercontent.com/klupp/opendata/master/mk/opstini-municipalities/municipalities_lq.geojson")
# mk_cities = mk_municipalities.dissolve(by='entity_mk', aggfunc='sum')                                
# # mk_cities['entity_mk'] = mk_cities.index
# mk_cities['id'] = mk_cities.index
# mk_cities
mk_municipalities

,geometry
0,"POLYGON ((21.44186 41.98734, 21.42585 41.98345..."
1,"POLYGON ((21.44986 41.99475, 21.44325 41.98318..."
2,"POLYGON ((21.45110 41.99906, 21.47937 42.08924..."
3,"POLYGON ((21.45050 42.01688, 21.40223 42.02671..."
4,"POLYGON ((21.45110 41.99906, 21.44904 42.02125..."
...,...
75,"POLYGON ((20.94588 41.88456, 20.94705 41.86669..."
76,"POLYGON ((20.77025 41.92561, 20.77281 41.92630..."
77,"POLYGON ((20.90203 41.67533, 20.85558 41.71699..."
78,"POLYGON ((20.93418 41.82703, 20.83071 41.82011..."


In [5]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age', 'source'], header=0)

infected_df

,city,infected_in,date,count,age,source
0,Скопје,Италија,2020-02-26,1,50-59,https://a1on.mk/macedonia/prv-sluchaj-na-zabol...
1,Дебар,Италија,2020-03-06,2,60-69,http://zdravstvo.gov.mk/dva-novi-potvrdeni-slu...
2,Дебар,Дебар,2020-03-09,2,60-69,https://sitel.com.mk/dva-novi-sluchai-na-koron...
3,Скопје,Италија,2020-03-09,1,50-59,https://makfax.com.mk/makedonija/%D1%84%D0%B8%...
4,Дебар,Дебар,2020-03-09,1,40-49,https://telma.com.mk/ushte-tri-novi-sluchai-na...
5,Дебар,Дебар,2020-03-11,2,60-69,https://a1on.mk/macedonia/dvajca-novi-zaboleni...
6,Дебар,Дебар,2020-03-13,4,50-59,https://civilmedia.mk/13-mart-chetiri-novi-slu...
7,Скопје,Шпанија,3/14/2020,1,непознато,https://www.magazin.mk/%D0%BA%D0%BE%D1%80%D0%B...
8,Дебар,Дебар,3/14/2020,5,непознато,https://makfax.com.mk/makedonija/%d0%b2%d0%b8%...
9,Скопје,Холандија,3/16/2020,1,50-59,https://kurir.mk/makedonija/vesti/pet-novi-slu...


In [6]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age', 'source'], header=0)
healed_df

,city,date,count,age,source
0,Скопје,2020-03-20,1,50-59,NaN


In [7]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age', 'source'])
dead_df

,city,date,count,age,source


In [8]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
covid_mk = pd.merge(infected_by_city, healded_by_city, on='city', how='outer', suffixes=['_infected', '_healed'])
covid_mk['count'] = covid_mk.count_infected
covid_mk = pd.merge(covid_mk, dead_by_city, on='city', how='outer', suffixes=['', '_dead'])
covid_mk.fillna(int(0), inplace=True)
covid_mk['count'] = covid_mk.count_infected - covid_mk.count_healed - covid_mk.count_dead
covid_mk = covid_mk.astype(int)
covid_mk

,count_infected,count_healed,count,count_dead
city,,,,
Гостивар,1,0,1,0
Дебар,30,0,30,0
Охрид,1,0,1,0
Скопје,35,1,34,0
Штип,1,0,1,0


In [9]:
covid_mk_cities = mk_cities.merge(covid_mk, left_on='id', right_on='city', how='left')
covid_mk_cities['geometry'] = covid_mk_cities.geometry.simplify(0.001)

In [10]:
covid_mk_cities['count'] = covid_mk_cities['count'].fillna(0).astype(int)
covid_mk_cities['count_infected'] = covid_mk_cities['count_infected'].fillna(0).astype(int)
covid_mk_cities['count_healed'] = covid_mk_cities['count_healed'].fillna(0).astype(int)
covid_mk_cities['count_dead'] = covid_mk_cities['count_dead'].fillna(0).astype(int)

In [11]:
scale = np.log2((covid_mk_cities['count'] / (covid_mk_cities['population'] / 2000) + 1))
idd = map(str, covid_mk_cities.index)
covid_active = dict(zip(idd, scale))
covid_mk_cities_geo_json = json.loads(covid_mk_cities.to_json())

In [12]:
covid_mk_cities[['geometry', 'id', 'count', 'count_infected', 'count_healed', 'count_dead', 'population']].to_file("covid.geojson", 'GeoJSON')
covid_mk_cities[['geometry', 'id', 'count', 'count_infected', 'count_healed', 'count_dead', 'population']].to_file("../../opendata/mk/covid19/maps/mk_municipalities_covid.geojson", 'GeoJSON')

In [50]:
hospital_data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=695931938&single=true&output=csv",
                           names=['hospital', 'latitude', 'longitude'], header=0)
hospital_data

,hospital,latitude,longitude
0,Клиника за инфективни болести Скопје,41.988005,21.425646
1,Општа болница Дебар,41.523717,20.523101
2,Клиника Жан Митрев Скопје,42.005272,21.403212


In [74]:
sick_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=480309052&single=true&output=csv", header=0,
                 names=['hospital', "date", "count", "age_range", 'origin', 'source'])
sick_hospitals

,hospital,date,count,age_range,origin,source
0,Клиника за инфективни болести Скопје,2020-02-26,1,50-59,Скопје,NaN
1,Клиника за инфективни болести Скопје,2020-03-06,2,60-69,Дебар,NaN
2,Клиника за инфективни болести Скопје,2020-03-09,2,60-69,NaN,NaN
3,Клиника за инфективни болести Скопје,2020-03-09,1,50-59,NaN,NaN
4,Клиника за инфективни болести Скопје,2020-03-09,1,40-49,NaN,NaN
5,Клиника за инфективни болести Скопје,2020-03-11,2,60-69,NaN,NaN
6,Клиника за инфективни болести Скопје,2020-03-13,4,50-59,NaN,NaN
7,Клиника за инфективни болести Скопје,3/14/2020,1,непознато,NaN,NaN
8,Општа болница Дебар,3/14/2020,5,непознато,Дебар,NaN
9,Клиника за инфективни болести Скопје,3/16/2020,1,50-59,NaN,NaN


In [75]:
recovered_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1488187986&single=true&output=csv", header=0,
                 names=['hospital', "date", "count", "age_range", 'source'])
recovered_hospitals

,hospital,date,count,age_range,source
0,Клиника за инфективни болести Скопје,2020-03-16,1,50-59,https://www.novamakedonija.com.mk/featured/%D0...


In [76]:
dead_hospitals = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=514503617&single=true&output=csv", header=0,
                 names=['hospital', "date", "count", "age_range", 'source'])
dead_hospitals

,hospital,date,count,age_range,source


In [77]:
sick_hospital = sick_hospitals.groupby('hospital').sum()
recovered_hospital = recovered_hospitals.groupby('hospital').sum()
dead_hospital = dead_hospitals.groupby('hospital').sum()

In [82]:
covid_hospitals = pd.merge(sick_hospital, recovered_hospital, on='hospital', how='outer', suffixes=['_infected', '_recovered'])
covid_hospitals = pd.merge(covid_hospitals, dead_hospital, on='hospital', how='outer', suffixes=['', '_dead'])
covid_hospitals.fillna(0, inplace=True)
covid_hospitals = pd.merge(covid_hospitals, hospital_data, on='hospital', how='inner')
covid_hospitals = covid_hospitals.set_index("hospital")
covid_hospitals['count_dead'] = covid_hospitals['count']
covid_hospitals.drop(['date', 'age_range', 'count', 'source', 'source_dead'], axis=1, inplace=True)
covid_hospitals['count_active'] = covid_hospitals.count_infected - covid_hospitals.count_recovered - covid_hospitals.count_dead
covid_hospitals.to_csv('../../opendata/mk/covid19/datasets/summary_by_hospital.csv')
gdf = geopandas.GeoDataFrame(covid_hospitals, geometry=geopandas.points_from_xy(covid_hospitals.longitude, covid_hospitals.latitude))
gdf.to_file("../../opendata/mk/covid19/maps/summary_hospital.geojson", 'GeoJSON')

In [15]:

colors = Choropleth(
    geo_data=covid_mk_cities_geo_json,
    choro_data=covid_active,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.5, 'weight': 0.05},
    hover_style={'fillColor': 'grey' , 'fillOpacity': 0.3, 'weight': 2, 'color': 'green'}
)

# def hover_handler(event=None, feature=None, id=None, properties=None):
#     label.value = properties['id'] + " " + str(properties['count'])

# geo_data.on_hover(hover_handler)
# label = ipyw.Label(layout=ipyw.Layout(width='100%'))

m = map_methods.get_macedonia_map()
hospitals = map_methods.marker_layer(d=covid_hospitals, 
                             cols=["count_x", "count_y", "count"], 
                             names=["Заразени", "Излечени" ,"Мртви"],
                            get_marker=get_marker,
                            hide_marker=hide_marker,
                            lng='Longitude',
                            lat='Latitude')
# layer_group = LayerGroup(layers=(colors, geo_data, ))
m.add_layer(colors)
m.add_layer(hospitals)
m.add_control(LayersControl())

m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…